In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import itertools
from functools import total_ordering
from numpy.random import default_rng

#our classes!!! :)
from ParticleSim import Disk
from ParticleSim import Expt

In [13]:
# SIMPLE TEST CELL FOR showAnimation
seed = 42
rng = default_rng(seed=seed) # instantiate the default random number generator
sqrtNumParts = 2

p1 = Disk([104.0, 10.0], [0.01, 30.0])
p2 = Disk([96.0, 190.0], [-0.01, -30.0])

e1 = Expt([p1, p2], dt=.04, tmax=8, animSpeed=100, updateGraphsEvery=1, 
          potentialType="Lenn", doCollisions=False, cool=False, L=200)

print("Initial KE:",e1.avgKE)
print("Initial PE:",e1.avgPE)
print("Initial Total E:",e1.avgE)
e1.showAnimation("seed = " + str(seed) + "     ")
print("Final KE:",e1.avgKE)
print("Final PE:",e1.avgPE)
print("Final Total E:",e1.avgE)

MovieWriter ffmpeg unavailable; using Pillow instead.


Initial KE: 1350.0001499999998
Initial PE: -3.741132312957726e-05
Initial Total E: 1350.0001125886768
initializing experiment... 
starting simulation... 
[████████████████████████████████]  
finished animating!
Final KE: 50501.16305222519
Final PE: -1.3661170796628936e-05
Final Total E: 50501.16303856402


In [ ]:
# TEST CELL FOR showAnimation
seed = 42
rng = default_rng(seed=seed) # instantiate the default random number generator
sqrtNumParts = 3

pPosGrid = list(itertools.product(np.linspace(20, 180,sqrtNumParts), np.linspace(20,180,sqrtNumParts)))
pVelGrid = list((rng.random(size=(sqrtNumParts**2,2))-.5)*50) #randomize initial velocities

particles = [Disk(pPosGrid[i], pVelGrid[i]) for i in range(sqrtNumParts**2)]

e1 = Expt(particles, dt=.1, tmax=10, animSpeed=7.5, updateGraphsEvery=5, 
          potentialType="Coul", doCollisions=False, cool=False)

print("Initial KE:",e1.avgKE)
print("Initial PE:",e1.avgPE)
print("Initial Total E:",e1.avgE)
e1.showAnimation("seed = " + str(seed) + "     ")
print("Final KE:",e1.avgKE)
print("Final PE:",e1.avgPE)
print("Final Total E:",e1.avgE)

In [ ]:
plt.close("all")

Feel free play some on your own!
https://github.com/cfloyd-hmc/CF-NH-Phys64
The ParticleSim class is pretty generic, and allows lots of different interactions and different systems.

Hard disk collisions, Lennard jones potential, Coulomb potential, particles with different radii, charges, ...